# 天氣資料爬蟲
許多資料由於網站資料不一致性，需要來回手動處理，因此有多個存檔斷點，亦無法直接全部執行。

Weather Crawling

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

from selenium import webdriver

# 開啟瀏覽器視窗(Chrome)
path = "C:/Users/brian/chromedriver-win64/chromedriver.exe"

driver = webdriver.Chrome(path)
def remove_matches(sentence, x, y):
    result = []
    i = 0
    result.append(sentence[i:x])
    i = y
    result.append(sentence[i:])
    return "".join(result)

column = ['Time', 'Temp', 'Weather', 'Weather 2', 'Wind', 'Humidity', 'Barometer(%)', 'Visibility']

mon_dict_num = {'Nov': 11, 'Dec': 12, 'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10}
mon_dict = {'Nov': 30, 'Dec': 31, 'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31}
year_dict = {'Jan': 2022, 'Feb': 2022, 'Mar': 2022, 'Apr': 2022, 'May': 2022, 'Jun': 2022, 'Jul': 2022, 'Aug': 2022, 'Sep': 2022, 'Oct': 2022, 'Nov': 2023, 'Dec': 2023}
newpax = []
for h in list(mon_dict.keys()):
    # if(h not in ["Nov"]): continue
    driver.get("https://www.timeanddate.com/weather/usa/new-york/historic?month={}&year={}".format(str(mon_dict_num[h]), year_dict[h])) 
    while True:
        try:
            pax = []
            for k in range(1, mon_dict[h]+ 1):
                # if(k not in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]): continue
                print(k, h)
                button = driver.find_element(By.LINK_TEXT, "{} {}".format(k, h))
                button.click()
                table = driver.find_element(By.ID, "wt-his") # 大框架
                table = table.find_elements(By.TAG_NAME, "tr")
                att = []
                for i in range(len(table)):
                    text = table[i].text
                    if ((i == 0 or i == 1) or i == len(table) - 1): continue
                    if i == 2: text = remove_matches(text, text.find(',') - 3, text.find('{}'.format(h)) + 3)
                    att = text.strip().split()
                    att = [value for value in att if value not in ['↑', 'km/h', '°C', 'mbar', 'km']]
                    if('No' in att):
                        num_index = att.index('No')
                        att[num_index] = 0
                        att.pop(num_index + 1)
                    if('N/A' in att):
                        num_index = att.index('N/A')
                        att[num_index] = 0
                    if (len(att) > 7):
                        for j in range((len(att) - 7)):
                            att[2] += ' '
                            att[2] += att[3]
                            att.pop(3)
                        condition = att[2].split('.')
                        att[2] = condition[0].strip()
                        att.insert(3, condition[1].strip())
                        if(att[3] == ''):
                            att[3] = np.nan
                    elif(len(att) == 7):
                        att[2] = att[2].replace('.', '')
                        att.insert(3, np.nan)
                    for f in [1, 4, 5, 6, 7]:
                        if(f == 5 and att[f] != 0):
                            att[f] = att[f].replace('%', '')
                        if(att[f] == 'N/A'):
                            att[f] = 0
                        att[f] = int(att[f])
                    att[0] = dt.datetime.strptime(('{}/{}/{}'.format(year_dict[h], mon_dict_num[h], k)).strip() + ' ' + att[0].strip(), '%Y/%m/%d %H:%M')
                    pax.append(att)
        except:
            continue
        else:
            break
    
    pd.DataFrame(pax, columns=column).to_csv('D:/作業/製造數據科學/Final Project/weather/{}{}.csv'.format(year_dict[h],str(mon_dict_num[h]).zfill(2)), index=False)
    # newpax.append(pax)
# pax = [w for x in newpax for w in x]
df_wea = pd.DataFrame(pax, columns=column)
# df_wea.to_csv('D:/作業/製造數據科學/Final Project/weather/weather.csv', index=False)

Transferring 30 descriptive weather conditions to dummy variables

In [11]:
import pandas as pd
import numpy as np
df = []
mon_dict_num = {'Nov': 11, 'Dec': 12, 'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10}
mon_dict = {'Nov': 30, 'Dec': 31, 'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31}
year_dict = {'Jan': 2022, 'Feb': 2022, 'Mar': 2022, 'Apr': 2022, 'May': 2022, 'Jun': 2022, 'Jul': 2022, 'Aug': 2022, 'Sep': 2022, 'Oct': 2022, 'Nov': 2021, 'Dec': 2021}
year_list = ['2021', '2021', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023', '2023', '2023', '2023', '2023', '2023', '2023', '2023', '2023', '2023', '2023']
month_list = [11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

for h, mon in enumerate(month_list):
    df.append(pd.read_csv('./weather/{}{}.csv'.format(year_list[h], str(mon).zfill(2))))
df = pd.concat(df, axis=0)
df = pd.read_csv('./weather/202312.csv')
filename = 'final_weather.csv'
weather = pd.get_dummies(df['Weather'], prefix='Weather').astype(int)
weather_2 = pd.get_dummies(df['Weather 2'], prefix='Weather').astype(int)
col = ['Weather_Heavy snow', 'Weather_Sunny', 'Weather_Light snow', 'Weather_Overcast', 'Weather_Dense fog', 'Weather_More clouds than sun', 'Weather_Heavy rain', 'Weather_Haze', 'Weather_Light freezing rain', 'Weather_Cloudy', 'Weather_Snow', 'Weather_Refreshingly cool', 'Weather_Mild', 'Weather_Broken clouds', 'Weather_Passing clouds', 'Weather_Fog', 'Weather_Partly cloudy', 'Weather_Partly sunny', 'Weather_Clear', 'Weather_Ice fog', 'Weather_Light rain', 'Weather_Cool', 'Weather_Quite cool', 'Weather_Warm', 'Weather_Rain', 'Weather_Mostly cloudy', 'Weather_Cold', 'Weather_Low clouds', 'Weather_Freezing rain', 'Weather_Scattered clouds']
for i in col:
    if(i in weather.columns and i in weather_2.columns):
        weather[i] = weather[i] + weather_2[i]
    elif(i in weather_2.columns):
        weather[i] = weather_2[i]
    elif(i in weather.columns):continue
    else:
        weather[i] = pd.Series(np.zeros(len(weather)))
weather = weather[col]
# print(weather.columns)
df[col] = weather[col]
df = df.drop(['Weather', 'Weather 2'], axis=1)

df.to_csv(filename, index=False)

In [12]:
df = pd.read_csv(filename)
import datetime as dt

new_df = []
for i in range(len(df)):
    date = dt.datetime.strptime(df["Time"].iloc[i], '%Y/%m/%d %H:%M')
    new_df.append('{}{}{} {}'.format(date.year, str(date.month).zfill(2), str(date.day).zfill(2), str(date.hour).zfill(2)))
df['Date'] = new_df
df.drop('Time', inplace=True, axis=1)

df.to_csv(filename, index = False)

Handling missing values

In [ ]:
import pandas as pd
import datetime as dt
df = pd.read_csv(filename)
# print(df.head())
# print(df)
new_df = []
print(len(df))
for i in range(len(df)):
    if(i != len(df) - 1):
        next = dt.datetime.strptime(df["Date"].iloc[i + 1], '%Y%m%d %H')
    else:
        break
    date = dt.datetime.strptime(df["Date"].iloc[i], '%Y%m%d %H')
    hour_num = int((next - date).total_seconds()/3600)-1
    # print(hour_num)
    if(hour_num <= 0): continue
    # print(hour_num)
    # print(date, next)
    for j in range(1, hour_num + 1):
        learn = (1/(hour_num + 1))
        df.loc[i + j*learn] = df.iloc[i].values
        if(date.hour + j > 23):
            target_string = '{}{}{} {}'.format(date.year, str(next.month).zfill(2), str(next.day).zfill(2), str(date.hour -24 + j).zfill(2))
            # print(target_string)
            df.loc[i + j*learn, 'Date'] = target_string
        else:
            target_string = '{}{}{} {}'.format(date.year, str(date.month).zfill(2), str(date.day).zfill(2), str(date.hour + j).zfill(2))
            # print(target_string)
            df.loc[i + j*learn, 'Date'] = target_string
df = df.sort_index().reset_index(drop=True)
print(len(df))
df = df.groupby('Date')
for i, ele in df:
    if(len(ele) > 1):
        # print(ele.iloc[0])
        temp = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, ele.iloc[0][35]]
        for j in range(len(ele)):
            for k, con in enumerate(list(ele.iloc[j])):
                if(k == 35): continue
                temp[k] += int(con)
        for k, con in enumerate(temp):
            if(k < 5):
                temp[k] = int(temp[k] / len(ele))
            elif(k != 35):
                temp[k] = min(1, temp[k])
    else:
        temp = list(ele.iloc[0])
    new_df.append(temp)

col = ['Temp', 'Wind', 'Humidity(%)', 'Barometer','Visibility', 'Weather_Heavy snow', 'Weather_Sunny', 'Weather_Light snow', 'Weather_Overcast', 'Weather_Dense fog', 'Weather_More clouds than sun', 'Weather_Heavy rain', 'Weather_Haze', 'Weather_Light freezing rain', 'Weather_Cloudy', 'Weather_Snow', 'Weather_Refreshingly cool', 'Weather_Mild', 'Weather_Broken clouds', 'Weather_Passing clouds', 'Weather_Fog', 'Weather_Partly cloudy', 'Weather_Partly sunny', 'Weather_Clear', 'Weather_Ice fog', 'Weather_Light rain', 'Weather_Cool', 'Weather_Quite cool', 'Weather_Warm', 'Weather_Rain', 'Weather_Mostly cloudy', 'Weather_Cold', 'Weather_Low clouds', 'Weather_Freezing rain', 'Weather_Scattered clouds', 'Date']

new_df = pd.DataFrame(new_df, columns=col)
new_df.to_csv(filename, index=False)
                

Check if every day is 24 hours

In [ ]:
df = pd.read_csv(filename)
year = []
hour = []
for i in range(len(df)):
    date = dt.datetime.strptime(df["Date"].iloc[i], '%Y%m%d %H')
    year.append('{}{}{}'.format(date.year, str(date.month).zfill(2), str(date.day).zfill(2)))
    hour.append('{}'.format(str(date.hour).zfill(2)))
df['Date'] = year
df['Hour'] = hour
iter = df.groupby('Date')
for i, ele in iter:
    if(len(ele) < 24):
        print(i, len(ele))